## CrossRef

In [ ]:
import requests, time, json, re, datetime, os, sys
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import numpy as np
import pandas as pd
from math import ceil
from datetime import date
# import crossRef_fun
# from crossRef_fun import getDataCiteInfo, getCrossRefCitations, filterCrossRefResults, mergeDFs, getPublicationInfo
# import exportCitationResultsToCsv
# from Results import convertCSVtoJSON


sys.path.insert(0, '..')


from citations_fun.getCrossRefCitations import getCrossRefCitations_byDates
from citations_fun.filterCrossRefResults import filterCrossRefResults


with open("results/nerc_datacite_dois.json") as f:
    nerc_datacite_dois = json.load(f)

nerc_datacite_dois_df = pd.DataFrame(nerc_datacite_dois)


In [3]:
# Code for getting results based on a date range

email = "matnic@ceh.ac.uk"
prefix = "10.5285"
start_date = "1990-01-01"
today = date.today()
end_date = today.strftime("%Y-%m-%d")
results_folder_path = "Results/"
results_folder_path_name = results_folder_path + "NERC_EDS_events_from_" + start_date + "_up_to_" + end_date

getCrossRefCitations_byDates(email, prefix, start_date, end_date, results_folder_path)

https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&rows=1000&obj-id.prefix=10.5285&from-occurred-date=1990-01-01&until-occurred-date=2025-07-15
Event Data query started...
API query complete  200
output file written to Results/event_data_10.5285_1990-01-01_2025-07-15.json
64304 events found
https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&cursor=2162cf3a-79be-4bd3-aaa7-1bdb682872b5&rows=1000&obj-id.prefix=10.5285&from-occurred-date=1990-01-01&until-occurred-date=2025-07-15
Event Data query started...
API query complete  200
output file written to Results/NERC_EDS_events_from_1990-01-01_up_to_2025-07-15/page0000.json
https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&cursor=ec36cf84-461c-40bb-8342-ad101145241a&rows=1000&obj-id.prefix=10.5285&from-occurred-date=1990-01-01&until-occurred-date=2025-07-15
Event Data query started...
API query complete  200
output file written to Results/NERC_EDS_events_from_1990-01-01_up_to_2025-07-15/page0001.json
https:/

In [4]:
# filter results 
crossRef_df_gbif_filtered2_deduplicated = filterCrossRefResults(results_folder_path_name)

In [5]:
crossRef_df_gbif_filtered2_deduplicated.head()

,relation_type_id,source_id,obj_id,subj_id,subj_work_type_id
17080,is-part-of,crossref,https://doi.org/10.5285/f98b053b-0cbc-6c23-e05...,https://doi.org/10.5194/os-21-1349-2025,journal-article
16927,references,crossref,https://doi.org/10.5285/27e550f2-81ff-6bf8-e06...,https://doi.org/10.1038/s41586-025-08921-3,journal-article
16928,references,crossref,https://doi.org/10.5285/2de087c9-cee3-87f4-e06...,https://doi.org/10.1038/s41586-025-08921-3,journal-article
16929,references,crossref,https://doi.org/10.5285/2392b266-126b-db3f-e06...,https://doi.org/10.1038/s41586-025-08921-3,journal-article
16930,references,crossref,https://doi.org/10.5285/03167D6F-A9CC-2E1A-E05...,https://doi.org/10.1007/978-3-031-76841-5_8,book-chapter


In [ ]:
# prefix = "https://doi.org/"
# data_doi_list = []
# for doi in crossRef_df_gbif_filtered2_deduplicated.obj_id:
#     data_doi_list.append(doi.replace(prefix, ""))


prefix = "https://doi.org/"
data_doi_list = [doi[len(prefix):] if doi.startswith(prefix) else doi for doi in crossRef_df_gbif_filtered2_deduplicated.obj_id]


crossRef_df_gbif_filtered2_deduplicated = crossRef_df_gbif_filtered2_deduplicated.drop(['obj_id'], axis = 1)
crossRef_df_gbif_filtered2_deduplicated['data_doi'] = data_doi_list




In [7]:
# match with nerc_datacite_dois_df to create df
nerc_datacite_dois_df.head()

,data_publisher,data_doi,data_title,data_dates,data_publication_year,data_authors,data_registered,data_page_number,data_self_link
0,Environmental Information Data Centre (EIDC),10.5285/36df3345-277d-4efa-9b98-9140dbea5c4f,Soil moisture data obtained from prototype neu...,"[{'date': '2025-06-30', 'dateType': 'Submitted...",2025.0,"[Stowell, P., Soler-Silva, M., Madari, B., Tor...",2025-07-04T15:59:22Z,1,https://api.datacite.org/dois?client-id=bl.ner...
1,Environmental Information Data Centre (EIDC),10.5285/a2c87e47-6f85-4bee-9a54-ecb1bb5b3573,Soil moisture data obtained from prototype neu...,"[{'date': '2025-06-30', 'dateType': 'Submitted...",2025.0,"[Stowell, P., Torre-Neto, A., Kamienski, C.]",2025-07-07T09:36:50Z,1,https://api.datacite.org/dois?client-id=bl.ner...
2,Environmental Information Data Centre (EIDC),10.5285/ec921bc2-5538-47ed-9e72-0d687b4ca4d3,UK maps of habitat suitability surfaces at 1km...,"[{'date': '2022-06-08', 'dateType': 'Submitted...",2022.0,"[Boyd, R., Pescott, O.L., Ball, S., Barber, T....",2022-06-09T14:45:58Z,1,https://api.datacite.org/dois?client-id=bl.ner...
3,Environmental Information Data Centre (EIDC),10.5285/a6012796-291c-4fd6-a7ef-6f6ed0a6cfa5,Daily and sub-daily hydrometeorological and so...,"[{'date': '2019-09-11', 'dateType': 'Submitted...",2019.0,"[Stanley, S., Antoniou, V., Ball, L.A., Bennet...",2019-09-16T08:42:42Z,1,https://api.datacite.org/dois?client-id=bl.ner...
4,Environmental Information Data Centre (EIDC),10.5285/39bed192-423a-4bbf-9b97-7d3da8aab445,Zooplankton taxa abundance from Windermere Nor...,"[{'date': '2025-07-02', 'dateType': 'Submitted...",2025.0,"[Thackeray, S.J., Mackay, E.B., Feuchtmayr, H.]",2025-07-03T13:54:30Z,1,https://api.datacite.org/dois?client-id=bl.ner...


In [21]:
# merge with data dois df

crossRef_events_df = crossRef_df_gbif_filtered2_deduplicated.merge(
    nerc_datacite_dois_df,
    left_on='data_doi',
    right_on='data_doi',
    how='left'  # left-join as crossRef_df_gbif_filtered2_deduplicated is a subset of nerc_datacite_dois_df
)

crossRef_events_df.head()

,relation_type_id,source_id,subj_id,subj_work_type_id,data_doi,data_publisher,data_title,data_dates,data_publication_year,data_authors,data_registered,data_page_number,data_self_link
0,is-part-of,crossref,https://doi.org/10.5194/os-21-1349-2025,journal-article,10.5285/f98b053b-0cbc-6c23-e053-6c86abc0af7b,British Oceanographic Data Centre (BODC),The GEBCO_2023 Grid - a continuous terrain mod...,"[{'date': '2022-06/2023-04', 'dateType': 'Coll...",2023.0,[GEBCO Bathymetric Compilation Group 2023],2023-04-20T16:28:53Z,38.0,https://api.datacite.org/dois?client-id=bl.ner...
1,references,crossref,https://doi.org/10.1038/s41586-025-08921-3,journal-article,10.5285/27e550f2-81ff-6bf8-e063-7086abc04f4f,British Oceanographic Data Centre (BODC),Benthic images recorded by a towed camera in 1...,"[{'date': '1978-06-16/1979-10-15', 'dateType':...",2024.0,"[Fleming, Bethany F, Simon-Lledo, Erik, Van Au...",2024-12-05T14:03:01Z,109.0,https://api.datacite.org/dois?client-id=bl.ner...
2,references,crossref,https://doi.org/10.1038/s41586-025-08921-3,journal-article,10.5285/2de087c9-cee3-87f4-e063-7086abc0f9a9,British Oceanographic Data Centre (BODC),Benthic images recorded by a Remotely Operated...,"[{'date': '2023-02-09/2023-03-09', 'dateType':...",2025.0,"[Fleming, Bethany F, Van Audenhaege, Loic, Sim...",2025-02-12T13:33:10Z,101.0,https://api.datacite.org/dois?client-id=bl.ner...
3,references,crossref,https://doi.org/10.1038/s41586-025-08921-3,journal-article,10.5285/2392b266-126b-db3f-e063-7086abc0fe00,British Oceanographic Data Centre (BODC),Benthic images recorded by a Remotely Operated...,"[{'date': '2023-02-09/2023-03-09', 'dateType':...",2024.0,"[Van Audenhaege, Loic, Simon-Lledo, Erik, Flem...",2024-10-18T15:14:53Z,114.0,https://api.datacite.org/dois?client-id=bl.ner...
4,references,crossref,https://doi.org/10.1007/978-3-031-76841-5_8,book-chapter,10.5285/03167D6F-A9CC-2E1A-E053-6C86ABC0753B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
crossRef_events_df.columns

Index(['relation_type_id', 'source_id', 'subj_id', 'subj_work_type_id',
       'data_doi', 'data_publisher', 'data_title', 'data_dates',
       'data_publication_year', 'data_authors', 'data_registered',
       'data_page_number', 'data_self_link'],
      dtype='object')

In [25]:
# rename columns

crossRef_events_df.rename(columns = {"source_id": "citation_source", "subj_id": "pub_doi", "subj_work_type_id": "pub_work_type"})

,relation_type_id,citation_source,pub_doi,pub_work_type,data_doi,data_publisher,data_title,data_dates,data_publication_year,data_authors,data_registered,data_page_number,data_self_link
0,is-part-of,crossref,https://doi.org/10.5194/os-21-1349-2025,journal-article,10.5285/f98b053b-0cbc-6c23-e053-6c86abc0af7b,British Oceanographic Data Centre (BODC),The GEBCO_2023 Grid - a continuous terrain mod...,"[{'date': '2022-06/2023-04', 'dateType': 'Coll...",2023.0,[GEBCO Bathymetric Compilation Group 2023],2023-04-20T16:28:53Z,38.0,https://api.datacite.org/dois?client-id=bl.ner...
1,references,crossref,https://doi.org/10.1038/s41586-025-08921-3,journal-article,10.5285/27e550f2-81ff-6bf8-e063-7086abc04f4f,British Oceanographic Data Centre (BODC),Benthic images recorded by a towed camera in 1...,"[{'date': '1978-06-16/1979-10-15', 'dateType':...",2024.0,"[Fleming, Bethany F, Simon-Lledo, Erik, Van Au...",2024-12-05T14:03:01Z,109.0,https://api.datacite.org/dois?client-id=bl.ner...
2,references,crossref,https://doi.org/10.1038/s41586-025-08921-3,journal-article,10.5285/2de087c9-cee3-87f4-e063-7086abc0f9a9,British Oceanographic Data Centre (BODC),Benthic images recorded by a Remotely Operated...,"[{'date': '2023-02-09/2023-03-09', 'dateType':...",2025.0,"[Fleming, Bethany F, Van Audenhaege, Loic, Sim...",2025-02-12T13:33:10Z,101.0,https://api.datacite.org/dois?client-id=bl.ner...
3,references,crossref,https://doi.org/10.1038/s41586-025-08921-3,journal-article,10.5285/2392b266-126b-db3f-e063-7086abc0fe00,British Oceanographic Data Centre (BODC),Benthic images recorded by a Remotely Operated...,"[{'date': '2023-02-09/2023-03-09', 'dateType':...",2024.0,"[Van Audenhaege, Loic, Simon-Lledo, Erik, Flem...",2024-10-18T15:14:53Z,114.0,https://api.datacite.org/dois?client-id=bl.ner...
4,references,crossref,https://doi.org/10.1007/978-3-031-76841-5_8,book-chapter,10.5285/03167D6F-A9CC-2E1A-E053-6C86ABC0753B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3177,references,crossref,https://doi.org/10.1007/s10533-017-0350-9,Info not given,10.5285/475520d5-bad9-4d84-e053-6c86abc0b01b,British Oceanographic Data Centre (BODC),Quantification of macrofaunal activity in the ...,"[{'date': '2015-06-23/2015-07-10', 'dateType':...",2017.0,"[Hale, Rachel, Wood, Christina L, Godbold, Jas...",2017-02-13T16:43:23Z,183.0,https://api.datacite.org/dois?client-id=bl.ner...
3178,references,crossref,https://doi.org/10.1007/s11269-018-1914-8,Info not given,10.5285/bad1514f-119e-44a4-8e1e-442735bb9797,Environmental Information Data Centre (EIDC),Future flows climate data,"[{'date': '2012-04-01', 'dateType': 'Submitted...",2012.0,"[Prudhomme, C., Dadson, S., Morris, D., Willia...",2012-03-20T16:22:01Z,7.0,https://api.datacite.org/dois?client-id=bl.ner...
3179,references,crossref,https://doi.org/10.1007/s11600-018-0165-7,Info not given,10.5285/5acfd143-1104-7b58-e053-6c86abc0d94btett,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3180,references,crossref,https://doi.org/10.1007/s00382-018-4234-z,Info not given,10.5285/4c7fdfa6-f176-4c58-acee-683d5e9d2ed5,Centre for Environmental Data Analysis (CEDA),CRU TS3.23: Climatic Research Unit (CRU) Time-...,"[{'date': '2015', 'dateType': 'Issued'}]",2015.0,"[Harris, Ian]",2015-11-09T22:39:13Z,189.0,https://api.datacite.org/dois?client-id=bl.ner...


### Output spreadsheet

In [ ]:
crossRef_df_processed_filename = "Results/Intermediate data/" + 'dataset_citation_publication_info_' + start_date + "_to_" + end_date + "_retrieved_" + (today.strftime("%d%m%Y")) + '.csv'
crossRef_df_gbif_filtered2_deduplicated.to_csv(crossRef_df_processed_filename, index = False)

with open("Results/Intermediate data/latest_results_crossRef.csv", 'w') as f:
    crossRef_df_gbif_filtered2_deduplicated.to_csv("Results/Intermediate data/latest_results_crossRef.csv",index = False)